# Content - OBSOLETE: This file was copied in the Notebook containing the answers
This notebook contains a script to derive soil properties according to:
- Gazetas, G. (1991). Formulas and charts for impedances of surface and embedded foundations. Journal of Geotechnical Engineering, 117(9), 1363–1381. https://doi.org/10.1061/(asce)0733-9410(1991)117:9(1363`
  
Input parameters are from `CIEM5220_SRE Project_Part 2_2025.pdf`

In [13]:
import numpy as np

In [14]:
# Soil Properties (indipendent of Student Number)
nu = 0.33      #[-] Poisson Ratio
c_u = 50e3  #[Pa] Undrained Shear Strength
v_s = 160      #[m/s] Velocity of shear waves in soil
rho = 1800     #[kg/m^3] Soil Density
beta = 0.05    #[-] Soil hysteretic (material) damping (arbitrarily chosen)

# Foundation dimensions (rectangular slab shallow foundation)
side_x = side_y = 5.81 #[m] As a first implementation the slab is as large as the base of the tower
depth = 0.3 #[m] height of foundation

# Structure
## Modal parameters from RFEM model (mode 2)
omega = 23.047 #[rad/s] Frequency of interest (Mode 2 - Main mode in X direction)


# Question 6 - Soil Structure Interaction

# Part A - Frequency Dependent Soil Stiffness Matrix

In [15]:
L = side_x/2                                  #[m] Half-length between center of the circumscribed rectangle and side X-direction (major direction)
B = side_y/2                                  #[m] Half-width between center of the circumscribed rectangle and side Y-direction (minor direction)
A_b = side_x*side_y                           #[m] Foundation surface area
I_bx = side_x*side_y**3/12                    #[m^4] Area moment of inertia about x-axis
I_by = side_x**3*side_y/12                    #[m^4] Area moment of inertia about y-axis
I_bz = side_x*side_y*(side_x**2+side_y**2)/12 #[m^4] Area moment of inertia about z-axis
G = c_u                                       #[Pa] Shear modulus
v_la = 3.4*v_s/(np.pi*(1-nu))                 #[m/s] Lymer's analog wave velocity (formula from cited paper)
chi = A_b/(4*L**2)                            #[-] Parameter from cited paper'"Part B - Notebook (Answers 3-6).ipynb"

print("\nINPUT RECAP:")
print(f"L: {L:.3e} m")
print(f"B: {B:.3e} m")
print(f"A_b: {A_b:.3e} m")
print(f"I_bx: {I_bx:.3e} m^4")
print(f"I_by: {I_by:.3e} m^4")
print(f"I_bz: {I_bz:.3e} m^4")
print(f"G: {G:.3e} Pa")
print(f"v_la: {v_la:.3e} m/s")
print(f"chi: {chi:.3e} -")

#Static Stiffness
K_z = (2*G*L/(1-nu))*(0.73+1.54*chi**0.75)
K_y = (2*G*L/(2-nu))*(2+2.50*chi**0.85)
K_x = K_y - (0.2/(0.75-nu))*G*L*(1-(B/L))
K_rx = (G/(1-nu))*I_bx**0.75*(L/B)**0.25*(2.4+0.5*(B/L))
K_ry = (3*G/(1-nu))*I_by**0.75*(L/B)**0.15
K_t = 3.5*G*I_bz**0.75*(B/L)**0.4*(I_bz/B**4)**0.2

# Dynamic Stiffness
a_0 = omega * B / v_s
print(f"a_0: {a_0:.3e} ")

## Table values
k_z_tab = 0.97         #(table Fig.2(a))
k_y_tab = 0.97         #(table Fig.2(b))
k_x_tab = 1            #(constant)
k_rx_tab = 1-0.20*a_0  #(Table 1)
k_ry_tab = 1-0.26*a_0  #(Table 1)
k_t_tab = 1-0.14*a_0   #(Table 1)

## Coefficients
K_z_dyn = K_z*k_z_tab
K_y_dyn = K_y*k_y_tab
K_x_dyn = K_x*k_x_tab
K_rx_dyn = K_rx*k_rx_tab
K_ry_dyn = K_ry*k_ry_tab
K_t_dyn = K_t*k_t_tab

print("\nDynamic Stiffness Coefficients:")
print(f"K_z_dyn: {K_z_dyn:.3e} N/m")
print(f"K_y_dyn: {K_y_dyn:.3e} N/m")
print(f"K_x_dyn: {K_x_dyn:.3e} N/m")
print(f"K_rx_dyn: {K_rx_dyn:.3e} N/m")
print(f"K_ry_dyn: {K_ry_dyn:.3e} N/m")
print(f"K_t_dyn: {K_t_dyn:.3e} N/m")

#  Radiation Dashpot Coefficients 
## Table values
c_z_tab = 0.93 #(table Fig.2(c))
c_y_tab = 0.88 #(table Fig.2(d))
c_rx_tab = 0.1 #(table Fig.2(e))
c_ry_tab = 0.1 #(table Fig.2(f))
c_t_tab = 0.1 #(table Fig.2(g))

## Coefficients
C_z = rho*v_la*A_b*c_z_tab
C_y = rho*v_s*A_b*c_y_tab
C_x = rho*v_s*A_b
C_rx = rho*v_la*I_bx*c_rx_tab
C_ry = rho*v_la*I_by*c_ry_tab
C_t = rho*v_s*I_bz*c_t_tab

## Total Damping (inclusive of materia damping)
C_z_tot = C_z+2*K_z_dyn*beta/omega
C_y_tot = C_y+2*K_y_dyn*beta/omega 
C_x_tot = C_x+2*K_x_dyn*beta/omega  
C_rx_tot = C_rx+2*K_rx_dyn*beta/omega 
C_ry_tot = C_ry+2*K_ry_dyn*beta/omega 
C_t_tot = C_t+2*K_t_dyn*beta/omega 
print("\nTotal Damping Coefficients")
print(f"C_z_tot: {C_z_tot:.3e} N/(m*s)")
print(f"C_y_tot: {C_y_tot:.3e} N/(m*s)")
print(f"C_x_tot: {C_x_tot:.3e} N/(m*s)")
print(f"C_rx_tot: {C_rx_tot:.3e} N/(m*s)")
print(f"C_ry_tot: {C_ry_tot:.3e} N/(m*s)")
print(f"C_t_tot: {C_t_tot:.3e} N/(m*s)")

# Dynamic Stiffness Matrix (inclusive of damping)
K_dyn_vec_6dof = [ K_x_dyn, K_y_dyn, K_z_dyn, K_rx_dyn, K_ry_dyn, K_t_dyn ]
C_tot_vec_6dof = [ C_x_tot, C_y_tot, C_z_tot, C_rx_tot, C_ry_tot, C_t_tot ]
K_tilde_6dof = np.diag([k+1j*omega*c for k,c in zip(K_dyn_vec_6dof,C_tot_vec_6dof)])
print("\nDynamic Stiffness Matrix (inclusive of damping):")
print(K_tilde_6dof)


INPUT RECAP:
L: 2.905e+00 m
B: 2.905e+00 m
A_b: 3.376e+01 m
I_bx: 9.496e+01 m^4
I_by: 9.496e+01 m^4
I_bz: 1.899e+02 m^4
G: 5.000e+04 Pa
v_la: 2.584e+02 m/s
chi: 1.000e+00 -
a_0: 4.184e-01 

Dynamic Stiffness Coefficients:
K_z_dyn: 9.547e+05 N/m
K_y_dyn: 7.593e+05 N/m
K_x_dyn: 7.828e+05 N/m
K_rx_dyn: 6.032e+06 N/m
K_ry_dyn: 6.069e+06 N/m
K_t_dyn: 1.025e+07 N/m

Total Damping Coefficients
C_z_tot: 1.461e+07 N/(m*s)
C_y_tot: 8.558e+06 N/(m*s)
C_x_tot: 9.725e+06 N/(m*s)
C_rx_tot: 4.444e+06 N/(m*s)
C_ry_tot: 4.444e+06 N/(m*s)
C_t_tot: 5.514e+06 N/(m*s)

Dynamic Stiffness Matrix (inclusive of damping):
[[  782784.43113772+2.24135607e+08j        0.        +0.00000000e+00j
         0.        +0.00000000e+00j        0.        +0.00000000e+00j
         0.        +0.00000000e+00j        0.        +0.00000000e+00j]
 [       0.        +0.00000000e+00j   759300.89820359+1.97246380e+08j
         0.        +0.00000000e+00j        0.        +0.00000000e+00j
         0.        +0.00000000e+00j        0

### Part B
As previously mentioned, mode 2 presents an Effective Modal Mass factor of 0.45, it was therefore considered the most appropriate to describe the system in a Single Degree of Freedom simplified system. 
The mass and moment of inertia of the SDoF system correspond to the effective modal mass in the translational x direction and rotational effective mass about the y-axis of the MDoF model. Both parameters can be extracted from the finite element model assembled in RFEM. The modal height can be computed as $h^*=\frac{\Gamma_{ry}}{\Gamma_t}$.
A modal damping ratio of $\zeta^*=0.048$ is assumed, therefore $c^*=2\zeta^*m^*\omega$. 
Finally the modal stiffness can be derived as $k^*= \omega^2 M^*$

A summary of the properties used in the conversion from MDoF to SDoF is presented in the following table:
|Parameter|Value|Origin|
|:-:|:-:|:-:|
|Modal mass| $1683\ kg$ |RFEM|
|Effective Modal Mass (x) > m| $3940\ kg$ |RFEM|
|Effective Modal Mass (r-y) > J| $371117\ kg\cdot m^2$ |RFEM|
|$\Gamma_t$| $2575$ |RFEM|
|$\Gamma_{ry}$| $24995$ |RFEM|
|$h^*$|$9.705\ m$|Computed|
|$c^*$| $3724\ kg \cdot rad/s$ |Computed|
|$k^*$| $2092787\ kg \cdot rad^2/s^2$ |Computed|

Taking into account the presence of the soil the SDOF system can therefore be represented as follows.

![SSI_system](Figures\SSI_system_diagram.png)

Where:

$u^r(t)$ relative displacement between top mass and foundation

$u_g^t(t) = u_g(t) + u_g^I(t)$ with: $u_g^t(t)$ total ground displacement, $u_g(t)$ free-field ground displacement, $u_g^I(t)$ ground displacement deriving from soil-structure interaction 

$\theta_g^t(t) = \cancel{\theta_g(t)} + \theta_g^I(t)$  with $\theta_g^t(t)$ total rocking motion of the ground, $\theta_g(t)$ free-field rocking motion (neglected), $\theta_g^I(t)$ rocking motion due to soil-structure interaction

The system can then be divided in two subsystems in order to analyze the interaction between soil and structure.

![SSI_subsystems](Figures\SSI_subsystems.png)

The translational equilibrium equation for the top mass can be formulated as:

$m\ddot{u^r}(t) + c\dot{u^r}(t) + ku^r(t) + m(\ddot{u}_g(t) + \ddot{u}_g^I(t) + h \ddot{\theta}_g^I(t)) = 0$

The translational equilibrium at soil-structure interface can be formulated as:

$m(\ddot{u}_g(t) + \ddot{u}_g^I(t) + \ddot{u^r}(t) + h\ddot{\theta}_g^I(t)) + m_f(\ddot{u}_g(t) + \ddot{u}_g^l(t)) = V_b(t)$

The rotational equilibrium at the  soil-structure interface can be formulated as:

$mh(\ddot{u}_g(t) + \ddot{u}_g^l(t) + \ddot{u^r}(t)) + (mh^2 + J + J_f)\ddot{\theta}_g^l(t) = M_b(t)$

Using a Fourier Transform the system can be converted to the frequency domain as follows:

$(-\omega^2m + i\omega c + k)\tilde{u^r}(\omega) - \omega^2m\tilde{u}_g^I(\omega) - \omega^2mh\tilde{\theta}_g^I(\omega) = -m\tilde{a_g}(\omega)$

$-\omega^2m\tilde{u^r}(\omega) - \omega^2(m + m_f)\tilde{u}_g^I(\omega) - \omega^2mh\tilde{\theta}_g^I(\omega) - \tilde{V}_b(\omega) = -(m + m_f)\tilde{a}_g(\omega)$

$-\omega^2mh\tilde{u^r}(\omega) - \omega^2mh\tilde{u}_g^I(\omega) - \omega^2(mh^2 + J + J_f)\tilde{\theta}_g^I(\omega) - \tilde{M}_b(\omega) = -mh\tilde{a}_g(\omega)$

Where: $\tilde{a_g} = \tilde{\ddot{u_g}}$

Moreover ${\tilde{V}_b(\omega)}^{SS-I} = - {\tilde{V}_b(\omega)}^{SS-II}$ and ${\tilde{M}_b(\omega)}^{SS-I} = - {\tilde{M}_b(\omega)}^{SS-II}$

$\tilde{V}_b(\omega)$ and $\tilde{M}_b(\omega)$ can therefore be derived from the soil stiffness matrix as:

$\tilde{V}_b(\omega) = - \tilde{k}_{xx}\tilde{u}_g^I(\omega)+\tilde{k}_{rx}\tilde{\theta}_g^I(\omega)$

$\tilde{M}_b(\omega) = - \tilde{k}_{rx}\tilde{u}_g^I(\omega)+\tilde{k}_{rr}\tilde{\theta}_g^I(\omega)$

Soil stiffness coefficients were derived following the methodology presented in Gazetas, G. (1991). Formulas and charts for impedances of surface and embedded foundations. Journal of Geotechnical Engineering, 117(9), 1363–1381.

After the final substitution, the system can therefore be rewritten in the following form:

$ (-\omega^2m + i\omega c + k)\tilde{u^r}(\omega) - \omega^2m\tilde{u}_g^I(\omega) - \omega^2mh\tilde{\theta}_g^I(\omega) = -m\tilde{a}_g(\omega) $

$ -\omega^2m\tilde{u^r}(\omega) + (\tilde{k}_{xx}(\omega) - \omega^2(m + m_f))\tilde{u}_g^I(\omega) + (\tilde{k}_{xr}(\omega) - \omega^2mh)\tilde{\theta}_g^I(\omega) = -(m + m_f)\tilde{a}_g(\omega) $

$ -\omega^2mh\tilde{u^r}(\omega) + (\tilde{k}_{rx}(\omega) - \omega^2mh)\tilde{u}_g^I(\omega) + (\tilde{k}_{rr}(\omega) - \omega^2(mh^2 + J + J_f))\tilde{\theta}_g^I(\omega) = -mh\tilde{a}_g(\omega) $

The algebraic system of equations can be solved for the three unknowns: $\tilde{u^r}(\omega)$, $\tilde{u}_g^I(\omega)$, $\tilde{\theta}_g^I(\omega)$

Finally an Inverse Fourier Transform can be performed to obtain  $u^r(t)$:

$ u^r(t) = \frac{1}{\pi} \int_{0}^{+\infty} \text{Re}({u}(\omega) \exp(i\omega t)) d\omega $



In [ ]:
# SDOF Model 
m_modal = 1683     #[kg] modal mass
M_x = 3940         #[kg] effective translational modal mass in the x direction
J_y = 371117       #[kg*m^2] effective rotational modal mass about the y-axis (moment of inertia)
gamma_t = 2575.4   #[-] translational participation factor in the x direction
gamma_ry = 24995.1 #[-] rotation participation factor about the y-axis 
zeta_star = 0.048  #[-] modal damping ration (mode 2, arbitrarily assumed)

h_star = gamma_ry/gamma_t
c_star = 2*zeta_star*m_modal*omega
k_star = omega**2*M_x

rho_f = 2400 #[kg/m^3] Density of concrete (arbitrarily chosen)
m_f = rho_f*side_x*side_y*depth #[kg] Mass of foundation
J_y_f = m_f*(side_x**2+depth**2)/12 #[kg*m^2] Moment of Inertia of foundation about the y-axis

# Soil Dynamic Stiffness Matrix (reduced to the relevant degrees of freedom: horizontal (x), vertical (z), rotational (yr))
K_dyn_vec = [K_x_dyn, K_ry_dyn]
C_tot_vec = [C_x_tot, K_ry_dyn]
K_d = np.diag([k+1j*omega*c for k,c in zip(K_dyn_vec,C_tot_vec)])

print(f"h_star: {h_star:.3f} m")
print(f"c_star: {c_star:.3f} kg*rad/s")
print(f"k_star: {k_star:.3f} kg*rad^2/s^2")

# Acceleration
a_g_tilde = 1 #[(m/s^2)/Hz] Acceleration in the frequency domain (taken as unitary as a first analysis#?) 

K_tilde_system = np.zeros((3,3), dtype=complex)
K_tilde_system[0, 0] = -omega**2*M_x + 1j*omega*c_star + k_star
K_tilde_system[1, 1] = K_d[0,0] - omega**2*(M_x+m_f)
K_tilde_system[2, 2] = K_d[1,1] - omega**2*(M_x*h_star**2+J_y+J_y_f)
K_tilde_system[0, 1] = K_tilde_system[1, 0] = -omega**2*M_x
K_tilde_system[0, 2] = K_tilde_system[2, 0] = -omega**2*M_x*h_star
K_tilde_system[1, 2] = K_tilde_system[2, 1] = K_d[0,1] - omega**2*M_x*h_star

F_tilde = np.array([-M_x*a_g_tilde, -(M_x+m_f)*a_g_tilde, -M_x*h_star*a_g_tilde])

u_r, _, _ = np.linalg.solve(K_tilde_system, F_tilde)



h_star: 9.705 m
c_star: 3723.658 kg*rad/s
k_star: 2092786.983 kg*rad^2/s^2
